In [2]:
from selenium.common import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.set_window_size(1024, 600)
driver.maximize_window()

driver.get("https://www.google.com/maps/place/Efrei+Paris+-+Campus+Gorki+-+Université+Paris-Panthéon-Assas/@48.7898571,2.3663896,17z/data=!3m1!4b1!4m6!3m5!1s0x47e673d0937ba945:0x63829c5f9fb306a!8m2!3d48.7898571!4d2.3685783!16s%2Fg%2F11nxd0nvh8?entry=ttu")

names = []
rating = []
dates = []
comments = []

try:
    # Wait for consent page to load
    accept_all_button = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, "[jsname='b3VHJd']"))
    )
    # Accept cookies because they are good
    accept_all_button.click()
except:
    print("An error occurred when trying to accept Google's terms of conditions.")

# get to reviews
try:
    reviews_button = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, "[aria-label^='Reviews for']"))
    )
    reviews_button.click()
except:
    print("An error occurred when trying to get to reviews.")

# scroll until the end
try:
    #.BHymgf.eiJcBe
    #.m6QErb.WNBkOb
    scrollable_div = driver.find_element(By.CSS_SELECTOR, ".m6QErb.DxyBCb.kA9KIf.dS8AEf")
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    print(last_height)
    while True:
        # scroll to the bottom
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div
        )
        # wait for the page to load
        time.sleep(2)

        # calculate new scroll height and compare with the last scroll height
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        print("newheight", new_height, "lastheight", last_height)
        if new_height == last_height:
            break
        last_height = new_height
except:
    print("Couldn't scroll...")

# solution to problem: get the coordinates (lat, long) between '@' and ',' and then use Google Maps api, enter coord, retrieve first element of search and get its place_id and get this url: https://www.google.com/maps/place/?q=place_id:"the_place_id"

# open the comments fully (see more)
try:
    # gather all see more buttons and click them
    all_see_more_buttons = driver.find_elements(By.CSS_SELECTOR, "[aria-label = 'See more'")
    for button in all_see_more_buttons:
        button.click()
except:
    print("Couldn't click all see more buttons.")

# extract text
try:
    comment_divs = driver.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium ")
    # there is necessarily a name, rating and a date but not comment
    for comment in comment_divs:
        names.append(comment.find_element(By.CLASS_NAME, "d4r55 ").text)
        rating.append(comment.find_element(By.CLASS_NAME, "kvMYJc").get_attribute("aria-label"))
        dates.append(comment.find_element(By.CLASS_NAME, "rsqaWe").text)
        # verify that there is comment and add, else add empty string
        text = ""
        try:
            myened_element = comment.find_element(By.CLASS_NAME, "MyEned")
            text = myened_element.find_element(By.CLASS_NAME, "wiI7pd").text
        except NoSuchElementException:
            text = ""
        comments.append(text)
except:
    print("Something happened when trying to parse reviews...")

finally:
    driver.quit()

zipped = list(zip(names, rating, dates, comments))
dictionary = {key: (v1, v2, v3) for key, v1, v2, v3 in zipped}

print(dictionary)

1955
newheight 2300 lastheight 1955
newheight 2300 lastheight 2300
{'Sinnay sinay': ('5 stars', 'a year ago', 'Hi-tec'), 'Luis Alba Sanchez': ('5 stars', 'a year ago', 'Excellent 👌 malgré des problèmes de wifi pour mon professeur …'), 'Super Bastodon (Bastodon)': ('2 stars', 'a year ago', ''), 'Axel Daoudongar': ('4 stars', 'a year ago', ''), 'Moe Lester': ('5 stars', '7 months ago', ''), 'Blaise': ('3 stars', '6 months ago', ''), 'César OMBREDANE': ('4 stars', 'a year ago', ''), 'jérémie-louis NDJE': ('4 stars', '5 months ago', ''), 'Tanguy Libert': ('5 stars', 'a year ago', ''), 'mendes voufo': ('5 stars', '9 months ago', ''), 'Jean Mhamoud': ('4 stars', '6 months ago', ''), 'Annabelle': ('5 stars', '4 months ago', ''), 'Victor Steimberg': ('5 stars', 'a year ago', '')}
